In [1]:
import pandas as pd
import numpy as np
import math
import pickle
from sklearn.pipeline import Pipeline, FeatureUnion, make_union, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import RobustScaler, Normalizer
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.dummy import DummyClassifier
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
from sklearn.externals import joblib
from CustomTransformers import ItemSelector, ArrayCaster


In [2]:
#Load corpora
sarcastic = pd.read_csv('../data/sarcastic.csv') 
non_sarcastic = pd.read_csv('../data/non_sarcastic.csv') 

In [3]:
                                                    #Pre-processing
       
#Remove '/s' annotation
sarcastic['body'] = sarcastic['body'].apply(lambda x: x.replace('/s', ''))

#Construct dataset
dataset = pd.DataFrame()
dataset['comment'] = sarcastic['body'].append(non_sarcastic['body'])
dataset['subreddit'] = pd.Series(sarcastic['subreddit']).append(pd.Series(non_sarcastic['subreddit']))
dataset['score'] = pd.Series(sarcastic['score']).append(pd.Series(non_sarcastic['score']))
sarcastic_labels = pd.Series(np.full(len(sarcastic), 1.0))
non_sarcastic_labels = pd.Series(np.full(len(non_sarcastic), 0.0))
dataset['target'] = sarcastic_labels.append(non_sarcastic_labels)

#Transform scores
dataset['score'] = dataset['score'].apply(lambda x: abs(x))

#Remove deleted
dataset = dataset[dataset.comment != '[deleted]']

#Reset index
dataset = dataset.reset_index()


In [4]:
                                                #Pre-processing
#Remove non-English comments

def calculate_ratios(text):
    languages_ratios = {}

    tokens = wordpunct_tokenize(text)
    words = [word.lower() for word in tokens]

    for language in stopwords.fileids():
        stopwords_set = set(stopwords.words(language))
        words_set = set(words)
        common_elements = words_set.intersection(stopwords_set)

        languages_ratios[language] = len(common_elements) 

    return languages_ratios


def is_English(text):
    ratios = calculate_ratios(text)

    most_rated_language = max(ratios, key=ratios.get)

    return True if most_rated_language == 'english' else False

dataset = dataset[dataset.apply(lambda x: is_English(x['comment']), axis=1)]

C:\Users\Rodrack\Anaconda\envs\snowflakes\lib\site-packages\ipykernel\__main__.py:13: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [5]:
#Convert numerical columns to numbers
dataset.apply(lambda x: pd.to_numeric(x, errors='ignore'))

#Split data (50%/30%/20%)
msk = np.random.rand(len(dataset)) < 0.5
training_data = dataset[msk]
test_data = dataset[~msk]
msk = np.random.rand(len(test_data)) < 0.3
validation_data = test_data[msk]
test_data = test_data[~msk]

In [20]:
                                                #Custom Transformers

# class ItemSelector(BaseEstimator, TransformerMixin):

#     def __init__(self, key):
#         self.key = key

#     def fit(self, x, y=None):
#         return self

#     def transform(self, data_dict):
#         return data_dict[self.key]
    
# class ArrayCaster(BaseEstimator, TransformerMixin):
#     def fit(self, x, y=None):
#         return self

#     def transform(self, data):
#         print data.shape
#         print np.transpose(np.matrix(data)).shape
#         return np.transpose(np.matrix(data))
    
    


In [6]:
#algorithm = MultinomialNB()
#algorithm = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42)
#algorithm = LogisticRegression(C=1e5)

n = 2
classifier = MultinomialNB()

ngram_count_vectorizer = Pipeline([
        ('extract', ItemSelector(key='comment')),
        ('ngram_tf_idf', TfidfVectorizer(ngram_range=(1, n), min_df=1))
    ])

subreddit_vectorizer = Pipeline([
        ('extract', ItemSelector(key='subreddit')),
        ('ngram_tf_idf', TfidfVectorizer(ngram_range=(1, n), min_df=1))
    ])

#N-grams and subreddit
pipeline2 = Pipeline([
  ('features', FeatureUnion([                   
    ('ngrams', ngram_count_vectorizer),
    ('subreddits', subreddit_vectorizer),
  ])),
  ('classifier', classifier)
])



text_clf = pipeline2.fit(training_data, training_data.target)
joblib.dump(text_clf, 'model.pkl') 
# text_clf.fit(training_data, training_data.target)
# predicted = text_clf.predict(validation_data)
# predicted2 = text_clf.predict(jere)

# predicted2

['model.pkl']

In [11]:
#                                                 #Model training

# #algorithm = MultinomialNB()
# #algorithm = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42)
# #algorithm = LogisticRegression(C=1e5)

n = 2
classifier = MultinomialNB()

ngram_count_vectorizer = Pipeline([
        ('extract', ItemSelector(key='comment')),
        ('ngram_tf_idf', TfidfVectorizer(ngram_range=(1, n), min_df=1))
    ])

# # pipeline0 = Pipeline([
# #          ('ngram_tf_idf', TfidfVectorizer(ngram_range=(1, n), min_df=1)),
# #          ('clf', classifier)
# #     ])

# # text_clf = pipeline0.fit(training_data.comment, training_data.target)
# # predicted = text_clf.predict(validation_data.comment)
# #np.mean(predicted == test_data.target) 

score_vectorizer = Pipeline([
        ('extract', ItemSelector(key='score')),
        ('caster', ArrayCaster()),
        ('scale', Normalizer())
    ])


subreddit_vectorizer = Pipeline([
        ('extract', ItemSelector(key='subreddit')),
        ('caster', ArrayCaster())
    ])

#Transform scores


#N-grams and score
# pipeline1 = Pipeline([
#   ('features', FeatureUnion([                   
#     ('ngrams', ngram_count_vectorizer),
#     ('normalized_scores', score_vectorizer),
#   ])),
#   ('classifier', classifier)
# ])

# # #N-grams and subreddit
pipeline2 = Pipeline([
  ('features', FeatureUnion([                   
    ('ngrams', ngram_count_vectorizer),
    ('subreddits', subreddit_vectorizer),
  ])),
  ('classifier', classifier)
])

# #N-grams, score and subreddit
# pipeline3 = Pipeline([
#   ('features', FeatureUnion([                   
#     ('ngrams', ngram_count_vectorizer),
#     ('normalized_scores', score_vectorizer),
#     ('subreddits', subreddit_vectorizer),
#   ])),
#   ('classifier', classifier)
# ])

text_clf = pipeline2.fit(training_data, training_data.target)
joblib.dump(text_clf, 'filename.pkl') 

#predicted = text_clf.predict(validation_data)
# np.mean(predicted == test_data.target) 

(25799L,)
(25799L, 1L)


TypeError: no supported conversion for types: (dtype('float64'), dtype('O'))

In [19]:
estimator =joblib.load("model.pkl")
jere = pd.DataFrame()
jere['comment'] = pd.Series('jjrjerjeje')
jere['subreddit'] = pd.Series('ForeverAlone')
name_category = estimator.predict(jere)

name_category[0]

1.0

In [9]:
                                                #Results
    
target_names = ['sarcastic', 'non-sarcastic']
#print(classification_report(validation_data.target, predicted, target_names=target_names))
accuracy_score(validation_data.target, predicted)
# tn, fp, fn, tp = confusion_matrix(validation_data.target, predicted).ravel()
# tn, fp, fn, tp

0.72686433063791556